# https://medium.com/@erikhallstrm/using-the-dynamicrnn-api-in-tensorflow-7237aba7f7ea

# Using the DynamicRNN API in TensorFlow (5/7)
In the previous guide we built a multi-layered LSTM RNN.
In this post we will speed it up by not splitting up our inputs and labels into a list, as done on line 41–42 in our code.
You may remove these rows where `inputs_series` and `labels_series` are declared.
Next change the `tf.nn.rnn` call on line 47 to the following:
```
states_series, current_state = tf.nn.dynamic_rnn(cell, tf.expand_dims(batchX_placeholder, -1), initial_state=rnn_tuple_state)
states_series = tf.reshape(states_series, [-1, state_size])
```
The `dynamic_rnn` function takes the batch inputs of shape `[batch_size, truncated_backprop_length, input_size]`, thus the addition of a single dimension on the end.
Output will be the last state of every layer in the network as an `LSTMStateTuple` stored `incurrent_state` as well as a tensor `states_series` with the shape `[batch_size, truncated_backprop_length, state_size]` containing the hidden state of the last layer across all time-steps.

![The states are not in lists anymore](pics/1_dh-jHqALFi7rnsEXYKc7Bw.png)

The tensor `states_series` is reshaped on the second row in the code sample above to shape `[batch_size*truncated_backprop_length, state_size]`, we will see the reason for this shortly.
You may read more about `dynamic_rnn` in the documentation.

Now input this two lines below the reshaping of the `states_series`.
```
logits = tf.matmul(states_series, W2) + b2 #Broadcasted addition
labels = tf.reshape(batchY_placeholder, [-1])
```
Notice that we are now only working with tensors, Python lists were a thing of the past.
The calculation of the logits and the labels are visualized below, notice the `state_series` variable that was reshaped earlier.
In TensorFlow reshaping is done in C-like index order.
It means that we read from the source tensor and “write” to the destination tensor with the last axis index changing fastest, and the first axis index changing slowest.
The result of the reshaping will be as visualized in the figure below, where similar colors denote the same time-step, and the vertical grouped spacing of elements denote different batches.

![Visualization of the calculations, similar color denote same time-step, vertical spacing denote new batch.](pics\1_mpS2qMofKbmr0nU1wm5tjQ.png)

Let’s go trough all the tensors in the figure above, first let’s start with the sizes.
We have that `batch_size=3`, `state_size=3`, `num_classes=2` and `truncated_backprop_length=3`.
The tensor `states_series` have shape `[batch_size*truncated_backprop_length, state_size]`, labels have shape `[batch_size*truncated_backprop_length]`, logits have shape `[batch_size*truncated_backprop_length, num_classes]`, W2 have shape `[state_size, num_classes]` and b2 have shape `[1, num_classes]`.
It can be a bit tricky to keep track of all the tensors, but drawing and visualizing with colors definitely helps.

Next calculate the predictions for the visualization:
```
logits_series = tf.unpack(tf.reshape(logits, [batch_size, truncated_backprop_length, num_classes]), axis=1)
predictions_series = [tf.nn.softmax(logit) for logit in logits_list]
```
Here we actually split the tensors into lists again.
This is perhaps not the best way to do it, but it’s quick and dirty, and the plot function is already expecting a list.

The `sparse_softmax_cross_entropy_with_logits` can take the shape of our tensors! Modify the losses calculation to this.
`losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)`

As we can read in the API the logits must have the shape `[batch_size, num_classes]` and labels must have the shape `[batch_size]`.
But now we are treating all time-steps as elements in our batch, so it will work out as we want.


In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

num_epochs = 5#100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length
num_layers = 3

def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

init_state = tf.placeholder(tf.float32, [num_layers, 2, batch_size, state_size])

state_per_layer_list = tf.unstack(init_state, axis=0)
rnn_tuple_state = tuple(
    [tf.nn.rnn_cell.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1])
     for idx in range(num_layers)]
)

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

# Forward passes
#cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
#cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers, state_is_tuple=True)

stacked_rnn = []
for _ in range(num_layers):
    stacked_rnn.append(tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True))
cell = tf.nn.rnn_cell.MultiRNNCell(stacked_rnn, state_is_tuple=True)

states_series, current_state = tf.nn.dynamic_rnn(cell, tf.expand_dims(batchX_placeholder, -1), initial_state=rnn_tuple_state)
states_series = tf.reshape(states_series, [-1, state_size])

logits = tf.matmul(states_series, W2) + b2 #Broadcasted addition
labels = tf.reshape(batchY_placeholder, [-1])

logits_series = tf.unstack(tf.reshape(logits, [batch_size, truncated_backprop_length, num_classes]), axis=1)
predictions_series = [tf.nn.softmax(logit) for logit in logits_series]

losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        x,y = generateData()

        _current_state = np.zeros((num_layers, 2, batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder: batchX,
                    batchY_placeholder: batchY,
                    init_state: _current_state
                })


            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Batch loss", _total_loss)
                plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()


New data, epoch 0
Step 0 Batch loss 0.6936618


Step 100 Batch loss 0.68458617


Step 200 Batch loss 0.58839816


Step 300 Batch loss 0.5865731


Step 400 Batch loss 0.39865032


...

Step 200 Batch loss 0.00036891134


Step 300 Batch loss 0.00038968885


Step 400 Batch loss 0.00034935639


Step 500 Batch loss 0.00026111535


Step 600 Batch loss 0.00033757355
